<a href="https://colab.research.google.com/github/Anastasiya9323/DES/blob/main/Klass_wine_zadanie301022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [375]:
import torch
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
torch.backends.cudnn.deterministic = True

In [376]:
wine = load_wine()
features = 13 # use 13 features

X_train, X_test, y_train, y_test = train_test_split(
    wine.data[:, :features], 
    wine.target, 
    test_size=0.3, 
    shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [377]:
class WineNet(torch.nn.Module):
    def __init__(self,n_input, n_hidden_neurons):
        super(WineNet, self).__init__()
        self.fc1 = torch.nn.Linear(n_input, n_hidden_neurons)
        self.activ1=torch.nn.Sigmoid()
        self.fc2=torch.nn.Linear(n_hidden_neurons,n_hidden_neurons)
        self.activ2=torch.nn.Sigmoid()
        self.fc3=torch.nn.Linear(n_hidden_neurons,3)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
n_input = features # choose number of input neurons
n_hidden = 15 # choose number of hidden neurons
wine_net = WineNet(n_input, n_hidden)

In [378]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(wine_net.parameters(), lr=1.0e-3)

In [379]:
batch_size =  50 # choose different batch sizes

for epoch in range(2000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = wine_net.forward(x_batch) 
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

tensor(0.4259)
tensor(0.3519)
tensor(0.7037)
tensor(0.8704)
tensor(0.8889)
tensor(0.8889)
tensor(0.8889)
tensor(0.8889)
tensor(0.8889)
tensor(0.9259)
tensor(0.9074)
tensor(0.9074)
tensor(0.9259)
tensor(0.9259)
tensor(0.9259)
tensor(0.9444)
tensor(0.9630)
tensor(0.9630)
tensor(0.9444)
tensor(0.9630)


In [390]:
print(wine_net.fc1.in_features, np.asarray((test_preds == y_test).float().mean()) > 0.8)
# need to get 13 True
print(len(wine.target[wine.target == 1]) / len(wine.target)) #количество элементов второго класса делим на количесвто элементов всех классов
print(np.unique(wine.target, return_counts=True)[1].max()/len(wine.target)) # берется максимальное количество элементов из всех трех классов
                                                                           # и делится на все элементы
print(max(np.bincount(wine.target)/wine.target.size)) #np.bincount(wine.target)/wine.target.size - массив из трех элементов 
                                                      #где каждый из элементов это количество элементов класса поделенное на общее количество элементов
                                                      # и из них выбирается максимальный
print((torch.LongTensor(wine.target) == torch.LongTensor([1]*wine.target.shape[0])).float().mean()) #хрен его знает что здесь творится помимо того что берут среднее

13 True
0.398876404494382
0.398876404494382
0.398876404494382
tensor(0.3989)
